In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

2024-04-12 00:27:28.873692: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 00:27:28.873750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 00:27:28.876052: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 00:27:28.884987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train = pd.read_csv('data_group.csv')
test = pd.read_csv('test_group.csv', delimiter='\t')

In [3]:
train

,Unnamed: 0,text,emotions
0,0,Dorian Gray with Rainbow Scarf #LoveWins (from...,happiness
1,1,@SelectShowcase @Tate_StIves ... Replace with ...,happiness
2,2,@Sofabsports thank you for following me back. ...,happiness
3,3,@britishmuseum @TudorHistory What a beautiful ...,happiness
4,4,@NationalGallery @ThePoldarkian I have always ...,happiness
...,...,...,...
624602,624602,What is that?,happiness
624603,624603,"Oh, they gave it to me at the sleep clinic, an...",happiness
624604,624604,"Well, are you asleep right now, Joe? 'Cause I ...",happiness
624605,624605,I know I don't have too! It tastes good.,happiness


In [4]:
train.columns

Index(['Unnamed: 0', 'text', 'emotions'], dtype='object')

In [5]:
train['emotions'].value_counts()

emotions
happiness    281576
sadness      143438
anger         92617
fear          55305
surprise      46357
disgust        5314
Name: count, dtype: int64

In [6]:
train

,Unnamed: 0,text,emotions
0,0,Dorian Gray with Rainbow Scarf #LoveWins (from...,happiness
1,1,@SelectShowcase @Tate_StIves ... Replace with ...,happiness
2,2,@Sofabsports thank you for following me back. ...,happiness
3,3,@britishmuseum @TudorHistory What a beautiful ...,happiness
4,4,@NationalGallery @ThePoldarkian I have always ...,happiness
...,...,...,...
624602,624602,What is that?,happiness
624603,624603,"Oh, they gave it to me at the sleep clinic, an...",happiness
624604,624604,"Well, are you asleep right now, Joe? 'Cause I ...",happiness
624605,624605,I know I don't have too! It tastes good.,happiness


In [7]:
X_train = train['text']
y_train = train['emotions']

X_test = test['sentence']

In [8]:
X_train

0         Dorian Gray with Rainbow Scarf #LoveWins (from...
1         @SelectShowcase @Tate_StIves ... Replace with ...
2         @Sofabsports thank you for following me back. ...
3         @britishmuseum @TudorHistory What a beautiful ...
4         @NationalGallery @ThePoldarkian I have always ...
                                ...                        
624602                                        What is that?
624603    Oh, they gave it to me at the sleep clinic, an...
624604    Well, are you asleep right now, Joe? 'Cause I ...
624605             I know I don't have too! It tastes good.
624606                                 Plus, you look cool.
Name: text, Length: 624607, dtype: object

In [9]:
X_test

0                   Girls are happy when they get flowers
1       His jaw dropped in disbelief when he saw the p...
2       Sometimes the ugly stench makes me wanna throw...
3       The foul odor from the garbage bin was disgust...
4       I can’t believe it, they lost the game in the ...
                              ...                        
1431    I feel guilty about the ice cream I stole as a...
1432    Her ability to recall obscure facts never ceas...
1433                     Your cat is so cute, I love him.
1434        She remained skeptical about the new product.
1435                               I love egg fried rice.
Name: sentence, Length: 1436, dtype: object

In [10]:
num_classes = 6
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

INFO: Using /tmp/tfhub_modules to cache modules.
2024-04-12 00:27:32.179968: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-04-12 00:27:32.180286: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43554 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:e1:00.0, compute capability: 8.6
INFO: Fingerprint not found. Saved model loading will continue.
INFO: path_and_singleprint metric could not be logged. Saved model loading will continue.
INFO: Fingerprint not found. Saved model loading will continue.
INFO: path_and_singleprint metric could not be logged. Saved model loading will continue.


In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

dropout = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
dense1 = tf.keras.layers.Dense(256, activation='relu', name="dense1")(dropout)

output = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(dense1)

model = tf.keras.Model(inputs=text_input, outputs = output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text_input[0][0]']          
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [14]:
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5


2024-04-12 00:27:46.969009: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2e30009480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-12 00:27:46.969044: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-04-12 00:27:46.978489: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-12 00:27:47.025935: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
I0000 00:00:1712881667.137200    4226 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7808/7808 [==============================] - ETA: 0s - loss: 1.1672 - accuracy: 0.5472


KeyboardInterrupt



In [ ]:
predictions = model.predict(X_test)

predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)

In [ ]:
np.unique(predictions)

In [ ]:
predictions = le.inverse_transform(predictions)

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'emotions': predictions})

submission

In [ ]:
submission['emotions'].unique()

In [ ]:
submission.to_csv('submission_transformer_different_architecture.csv', index=False)